<a href="https://colab.research.google.com/github/fintechsteve/modeling-volatility/blob/master/Part_04_Understanding_Turbulence_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 05 (a): Understanding GARCH Estimation

### In this section you will:


*   Learn about GARCH estimation and its use in investment management
*   Fit a GARCH Model to currency data
*   Use a GARCH Model forecast volatility


### Intro

GARCH (Generalized Autoregressive Conditional Heteroskedastic) Estimation is used to model the variance of a time series.  Applied to investment management, a GARCH model can be used to model, and therefore predict, asset volatility.  Such models were proposed in "[Generalized Autoregressive Conditional Heteroskedasticity](https://doi.org/10.1016/0304-4076%2886%2990063-1)"

Before we define a GARCH model, consider first an ARCH (Autoregressive Conditional Heteroskedastic) model.  This model assumes that the variance at time $t$ ($\sigma^2_t$) depends linearly on the error $\epsilon_t$,  at a fixed number ($q$) of previous time periods.  When ${q = 1}$ and assuming that the time series ${r_t}$  is stationary with constant mean ${\mu}$ we have the ARCH(1) model:

\begin{eqnarray*}
   r_t    & = & \mu + \epsilon_t \\
   \epsilon_t & = & \sigma_t e_t \\
   \sigma^2_t & = & \omega + \alpha \epsilon_{t-1}^2
\end{eqnarray*}

More generally for ${q > 1}$ we have ARCH(q):

\begin{eqnarray*}
   \sigma^2_t & = & \omega + \sum_{i=1}^{q}\alpha_i \epsilon_{t-i}^2 
\end{eqnarray*}

The ARCH model captures the idea that the volatility of the series may change through time.  For example, a financial market may exhibit a period of low volatility followed by a period of high volatility. A time series that exhibits periods with increased (or decreased) volatility is called conditional heteroskedastic. 

The ARCH model may be extended to a GARCH model by adding linear dependence on a fixed number ($p$) of past variances.  For ${p = 1}$  and ${q = 1}$ the GARCH(1,1) model is

\begin{eqnarray*}
   r_t    & = & \mu + \epsilon_t \\
   \epsilon_t & = & \sigma_t e_t \\
   \sigma^2_t & = & \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma^2_{t-1}
\end{eqnarray*}

More generally for ${p > 1}$ and ${q > 1}$ we have GARCH(p,q):

\begin{eqnarray*}
   \sigma^2_t & = & \omega + \sum_{i=1}^{q}\alpha_i \epsilon_{t-i}^2 + \sum_{j=1}^{p}\beta_j \sigma^2_{t-j}
\end{eqnarray*}

## Import all necessary libraries

For this piece, we will need the following packages to be available to our environment:

*   Pickle (For loading the data)
*   Numpy and Pandas (For data manipulation)
*   Arch (For GARCH estimation)
*   Matplotlib and Seaborn(For time series visualization)

If the packages are not available, install the with "pip install X" or "conda install -c bashtage arch"

In [ ]:
import numpy as np, pandas as pd
import pickle
from arch import arch_model
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Read in data from previously stored returns.pkl file



In [ ]:
with open('./returns.pkl', 'rb') as f:
    returns = pickle.load(f)
    f.close()
returns.head(10)

### Calculate DXY Index

In this part we scale the DXY Index to avoid numerical instabilities in the model fitting that may arise for inputs near zero.

In [ ]:
dxy_weight = [0, 0.119, 0.036, 0, 0.136, 0.576, 0, 0, 0.091]
dxy = returns.dot(dxy_weight)
scaled_dxy = 10000*dxy
scaled_dxy.head(10)

### Estimate a GARCH(1,1) model for the DXY Index

In [ ]:
am = arch_model(scaled_dxy, mean = 'Constant', vol='GARCH', p=1, q=1)
res = am.fit(update_freq=5)
print(res.summary())

The above model corresponds to ${\alpha  = .0422}$,  ${\beta = .9552}$, ${\omega = 9.5954e-04}$ and ${\mu = 1.8657e-03}$ in the frame work described above

\begin{eqnarray}
   r_t    & = & \mu + \epsilon_t \\
   \epsilon_t & = & \sigma_t e_t \\
   \sigma^2_t & = & \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma^2_{t-1}
\end{eqnarray}


### Forecast volatility for the DXY Index

Now we can use this model to forecast the future variance (volatility).

In [ ]:
forecasts = res.forecast(horizon = 1)
print(forecasts.variance.index[-1])
print(forecasts.variance.values[-1, -1]/10000)

Next we use a rolling window to forecast the one day volatility.  For each day we use the prior 260 days to fit a GARCH model and forecast a one day horizon.

In [ ]:
n_days = 100
lookback_window = 260
horizon = 1
f = []
for i in range(1,n_days+1):
    am = arch_model(scaled_dxy[-i-lookback_window:-i], mean = 'Constant', vol='GARCH', p=1, q=1)
    res = am.fit(disp='off')
    forecasts = res.forecast(horizon = horizon)
    f.append({'var': forecasts.variance.iloc[-1, -1]/10000, 'date':forecasts.variance.index[-1]})
    
df_vol_pred = pd.DataFrame(f[::-1])
df_vol_pred.head(10)

In [ ]:
sns.lineplot(x='date',y='var',data=df_vol_pred)

Next we can compare with daily volatility.

In [ ]:
dxy_sq = (dxy**2)
plt.plot(dxy_sq.values[-n_days:])

Now we can compute a rolling one day volatility forecast for the entire period.  We compute forcasts for ${p}$ = 1, 2, and 3 and ${q}$ = 1, 2, and 3.  Note: this step is time consuming.  You may wish to load presaved data rather than running it live.

In [ ]:
n_days = dxy.size-260
lookback_window = 260
horizon = 1
f = []
for p in range(1,4):
    for q in range(1,4):
        for i in range(n_days+1, 1, -1):
            am = arch_model(scaled_dxy[-i-lookback_window:-i], mean = 'Constant', vol='GARCH', p=p, q=q)
            res = am.fit(disp='off')
            forecasts = res.forecast(horizon = horizon)
            f.append({'var': forecasts.variance.iloc[-1, -1]/10000, 'date':forecasts.variance.index[-1], 'p':p, 'q':q})
            
df_vol_pred = pd.DataFrame(f)
df_vol_pred.to_pickle('./vol_pred.pkl')